In [91]:
import os
from glob import glob
from tqdm import tqdm
import importlib

import matplotlib
%matplotlib inline

import Galfit
importlib.reload(Galfit)

<module 'Galfit' from '/Users/jakesummers7200/JWST/PEARLS/GALAXY_HALOS_PAPER/GalPy/Galfit.py'>

In [92]:
# PARAMETERS THAT CHANGE FOR DIFFERENT DATASETS AND OBJECTS *******
field = 'clio'
date = '20230718'
img_size = 100 # (max = 333/2)
psf_size = 100 # usually 100 for both sizes
star = False

In [93]:
#directories
image_dir = f'/Users/jakesummers7200/JWST/PEARLS/GALAXY_HALOS_PAPER/{field}/mosaics'
object_dir = f'/Users/jakesummers7200/JWST/PEARLS/GALAXY_HALOS_PAPER/{field}/objects'
psf_dir =  f'/Users/jakesummers7200/JWST/PEARLS/GALAXY_HALOS_PAPER/{field}/psfs'

# catalog
cat_dir = f'/Users/jakesummers7200/JWST/PEARLS/GALAXY_HALOS_PAPER/{field}/cats'
master_cat = os.path.join(cat_dir, f'{field.upper()}_total_cat_extinction.txt')
cats = glob(os.path.join(cat_dir, f'{field.upper()}_F*_cat.txt'))
cats.sort()

# i2d images
images = glob(os.path.join(image_dir, f'mosaic_{field}_nircam_*_30mas_{date}_i2d.fits'))
images.sort()

In [94]:
# initialize galfit and create psf cutouts
gal = Galfit.Galfit(field, images, cats, master_cat, img_size=img_size, psf_size=psf_size, star=star,
             image_dir=image_dir, object_dir=object_dir, psf_dir=psf_dir, cat_dir=cat_dir)

Creating PSF cutouts...: 100%|████████████████████| 6/6 [00:00<00:00, 53.42it/s]


In [95]:
clio_cool_objects = [3061, 4143, 417, 64, 19, 7, 1645, 1543, 4858, 4763, 4385, 1062, 988, 530, 767, 739, 629, 433, 475, 3906, 1620, 4717, 4741, 4764, 4509, 1315, 1591, 627, 1179, 383]
g191_cool_objects = [5263, 2635, 1949, 1503, 677, 874, 1509, 422, 2013, 1780, 1763, 2004, 2764, 3003, 4560, 5033, 4962, 4795, 5300, 4840, 4827, 4763, 5359, 6175, 8994, 8637]


In [241]:
obj_id = 383 

gal.make_object(obj_id, overwrite=True)

Creating object cutouts...: 100%|█████████████████| 6/6 [00:00<00:00, 14.04it/s]


In [242]:
# run
for im in tqdm(images, desc='Running GALFIT...'):
    filt = im.split('_')[-4]
    gal.add_component(obj_id, filt, 'sersic', bulge=True)
    gal.add_component(obj_id, filt, 'sersic', bulge=False)
    
    gal.add_constraints(obj_id, filt, '1-2  mag  -2 2') # constrain bulge and disk to be within a few magnitues
    gal.add_constraints(obj_id, filt, '1_2  x    ratio') # keeps x position fixed between components
    gal.add_constraints(obj_id, filt, '1_2  y    ratio') # keeps y position fixed between components
    gal.add_constraints(obj_id, filt, '1    n    2 to 10') # fix bulge sersic index between 2 and 10
    gal.add_constraints(obj_id, filt, '2    n    0.4 to 3') # fix disk sersic index between 0.4 to 5
    gal.add_constraints(obj_id, filt, '1-2  n    0.1 10') # force bulge sersic > disk sersic
    gal.add_constraints(obj_id, filt, '1_2  pa   ratio') # keeps position angle fixed between components
    gal.add_constraints(obj_id, filt, '1/2  9    0.7 1.3') # keeps axis ratio similar between components
    gal.add_constraints(obj_id, filt, '2/1  re   0.2 5') # fix disk/bulge half light radius to be ratio 1 to 8
    
    gal.run(obj_id, filt, num='01', overwrite=True)

Running GALFIT...:   0%|                                  | 0/6 [00:00<?, ?it/s]


GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: galfit.feedme


# IMAGE and GALFIT CONTROL PARAMETERS
A) obj_383_f090w_sci.fits      # Input data image (FITS file)
B) obj_383_f090w_imgblock01.fits      # Output data image block
C) obj_383_f090w_err.fits      # Sigma image name (made from data if blank or "none") 
D) ../../../psfs/f090w_webbpsf_cutout.fits          # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) obj_383_f090w_mask.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) constraints.txt      # File with parameter constraints (ASCII file) 
H) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)
I) 200    200          # Size of the convolution box (x y)
J) 28.087              # Magnitude photometric zeropoint 
K) 0.030  0.030        # Plate scale (dx dy)   [arcsec per pixel]
O) regular             # Display type (regular, curses, both)
P) 0                   # Choose: 0

Running GALFIT...:  17%|████▎                     | 1/6 [00:50<04:14, 50.96s/it]



 The fitting ended because the maximum number of iterations 
 has been reached, not necessarily because of convergence!



Fit summary is now being saved into `fit.log'.

         parameters enclosed in *...*.  The solution is likely to be not 
         reliable.  Rerun fit by holding problematic parameters fixed 
         temporarily or setting them to other initial values.



GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: galfit.feedme


# IMAGE and GALFIT CONTROL PARAMETERS
A) obj_383_f150w_sci.fits      # Input data image (FITS file)
B) obj_383_f150w_imgblock01.fits      # Output data image block
C) obj_383_f150w_err.fits      # Sigma image name (made from data if blank or "none") 
D) ../../../psfs/f150w_webbpsf_cutout.fits          # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) obj_383_f150w_mask.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) constraints.txt      # File with

Running GALFIT...:  33%|████████▋                 | 2/6 [01:41<03:21, 50.44s/it]

...



Fit summary is now being saved into `fit.log'.


GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: galfit.feedme


# IMAGE and GALFIT CONTROL PARAMETERS
A) obj_383_f356w_sci.fits      # Input data image (FITS file)
B) obj_383_f356w_imgblock01.fits      # Output data image block
C) obj_383_f356w_err.fits      # Sigma image name (made from data if blank or "none") 
D) ../../../psfs/f356w_webbpsf_cutout.fits          # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) obj_383_f356w_mask.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) constraints.txt      # File with parameter constraints (ASCII file) 
H) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)
I) 200    200          # Size of the convolution box (x y)
J) 28.087              # Magnitude photometric zeropoint 
K) 0.030  0.030        # Plate scale (dx dy)   [arcsec per pixel]
O) regular             # Display type (regula

Running GALFIT...:  83%|█████████████████████▋    | 5/6 [03:03<00:32, 32.67s/it]



Fit summary is now being saved into `fit.log'.


GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: galfit.feedme


# IMAGE and GALFIT CONTROL PARAMETERS
A) obj_383_f444w_sci.fits      # Input data image (FITS file)
B) obj_383_f444w_imgblock01.fits      # Output data image block
C) obj_383_f444w_err.fits      # Sigma image name (made from data if blank or "none") 
D) ../../../psfs/f444w_webbpsf_cutout.fits          # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) obj_383_f444w_mask.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) constraints.txt      # File with parameter constraints (ASCII file) 
H) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)
I) 200    200          # Size of the convolution box (x y)
J) 28.087              # Magnitude photometric zeropoint 
K) 0.030  0.030        # Plate scale (dx dy)   [arcsec per pixel]
O) regular             # Display type (regula

Running GALFIT...: 100%|██████████████████████████| 6/6 [03:52<00:00, 38.80s/it]



 The fitting ended because the maximum number of iterations 
 has been reached, not necessarily because of convergence!



Fit summary is now being saved into `fit.log'.



In [243]:
# save template images
for im in tqdm(images, desc='Getting templates...'):
    filt = im.split('_')[-4]
    gal.create_templates(obj_id, filt, num='02', oldnum='01', overwrite=True)

Getting templates...:  17%|███▊                   | 1/6 [00:00<00:03,  1.57it/s]


GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: galfit.feedme


# IMAGE and GALFIT CONTROL PARAMETERS
A) obj_383_f090w_sci.fits      # Input data image (FITS file)
B) obj_383_f090w_imgblock02.fits      # Output data image block
C) obj_383_f090w_err.fits      # Sigma image name (made from data if blank or "none") 
D) ../../../psfs/f090w_webbpsf_cutout.fits #        # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) obj_383_f090w_mask.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) constraints.txt      # File with parameter constraints (ASCII file) 
H) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)
I) 200    200          # Size of the convolution box (x y)
J) 28.087              # Magnitude photometric zeropoint 
K) 0.030  0.030        # Plate scale (dx dy)   [arcsec per pixel]
O) regular             # Display type (regular, curses, both)
P) 3                   # Choose: 0

Getting templates...:  33%|███████▋               | 2/6 [00:01<00:02,  1.58it/s]


GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: galfit.feedme


# IMAGE and GALFIT CONTROL PARAMETERS
A) obj_383_f150w_sci.fits      # Input data image (FITS file)
B) obj_383_f150w_imgblock02.fits      # Output data image block
C) obj_383_f150w_err.fits      # Sigma image name (made from data if blank or "none") 
D) ../../../psfs/f150w_webbpsf_cutout.fits #        # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) obj_383_f150w_mask.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) constraints.txt      # File with parameter constraints (ASCII file) 
H) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)
I) 200    200          # Size of the convolution box (x y)
J) 28.087              # Magnitude photometric zeropoint 
K) 0.030  0.030        # Plate scale (dx dy)   [arcsec per pixel]
O) regular             # Display type (regular, curses, both)
P) 3                   # Choose: 0

Getting templates...:  50%|███████████▌           | 3/6 [00:01<00:01,  1.60it/s]

...


GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: galfit.feedme


# IMAGE and GALFIT CONTROL PARAMETERS
A) obj_383_f277w_sci.fits      # Input data image (FITS file)
B) obj_383_f277w_imgblock02.fits      # Output data image block
C) obj_383_f277w_err.fits      # Sigma image name (made from data if blank or "none") 
D) ../../../psfs/f277w_webbpsf_cutout.fits #        # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) obj_383_f277w_mask.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) constraints.txt      # File with parameter constraints (ASCII file) 
H) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)
I) 200    200          # Size of the convolution box (x y)
J) 28.087              # Magnitude photometric zeropoint 
K) 0.030  0.030        # Plate scale (dx dy)   [arcsec per pixel]
O) regular             # Display type (regular, curses, both)
P) 3                   # Choose: 0

Getting templates...:  83%|███████████████████▏   | 5/6 [00:03<00:00,  1.60it/s]


GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: galfit.feedme


# IMAGE and GALFIT CONTROL PARAMETERS
A) obj_383_f356w_sci.fits      # Input data image (FITS file)
B) obj_383_f356w_imgblock02.fits      # Output data image block
C) obj_383_f356w_err.fits      # Sigma image name (made from data if blank or "none") 
D) ../../../psfs/f356w_webbpsf_cutout.fits #        # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) obj_383_f356w_mask.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) constraints.txt      # File with parameter constraints (ASCII file) 
H) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)
I) 200    200          # Size of the convolution box (x y)
J) 28.087              # Magnitude photometric zeropoint 
K) 0.030  0.030        # Plate scale (dx dy)   [arcsec per pixel]
O) regular             # Display type (regular, curses, both)
P) 3                   # Choose: 0

Getting templates...: 100%|███████████████████████| 6/6 [00:03<00:00,  1.61it/s]


GALFIT Version 3.0.5 -- Apr. 23, 2013




#  Input menu file: galfit.feedme


# IMAGE and GALFIT CONTROL PARAMETERS
A) obj_383_f444w_sci.fits      # Input data image (FITS file)
B) obj_383_f444w_imgblock02.fits      # Output data image block
C) obj_383_f444w_err.fits      # Sigma image name (made from data if blank or "none") 
D) ../../../psfs/f444w_webbpsf_cutout.fits #        # Input PSF image and (optional) diffusion kernel
E) 1                   # PSF fine sampling factor relative to data 
F) obj_383_f444w_mask.fits      # Bad pixel mask (FITS image or ASCII coord list)
G) constraints.txt      # File with parameter constraints (ASCII file) 
H) 1    200  1    200  # Image region to fit (xmin xmax ymin ymax)
I) 200    200          # Size of the convolution box (x y)
J) 28.087              # Magnitude photometric zeropoint 
K) 0.030  0.030        # Plate scale (dx dy)   [arcsec per pixel]
O) regular             # Display type (regular, curses, both)
P) 3                   # Choose: 0

In [244]:
# make galfit plots
gal.plot_all(obj_id, field, num='01', s=50)

Making plots...: 100%|████████████████████████████| 6/6 [00:04<00:00,  1.31it/s]


In [245]:
# make aperture plots
gal.plot_all_apertures(obj_id, field, num='01', components=['bulge', 'disk'], frac_hl=2)

Making plots...:  17%|████▋                       | 1/6 [00:00<00:00,  7.05it/s]WARNING: No meaningful fit was possible. [photutils.isophote.ellipse]


fatal error 1!!!
fatal error 1!!!


Making plots...:  67%|██████████████████▋         | 4/6 [00:18<00:10,  5.29s/it]WARNING: No meaningful fit was possible. [photutils.isophote.ellipse]


fatal error 1!!!


Making plots...: 100%|████████████████████████████| 6/6 [00:27<00:00,  4.51s/it]
